In [ ]:
#Old Wooden Ships
#extracting data from the spreadsheet
import pandas as pd

#step 1: read in log csv
def get_log_csv():
    csvlogs = pd.read_csv('CLIWOC15.csv', low_memory=False)
    #convert all ship names to upper case
    csvlogs['ShipName'] = map(lambda x: x.upper(), csvlogs['ShipName'])
    
    #we only need ShipName, VoyageFrom, VoyageTo, Nationality, Year
    return csvlogs[['ShipName', 'VoyageFrom', 'VoyageTo', 'Nationality', 'Year']]
    

#Step 2:  Identify unique ships and select one record per ship

def extract_records():
    csvlogs = get_log_csv()
    uniques = pd.Series(csvlogs['ShipName']).unique()
    uIndex = 0
    id=0
    newframe = pd.DataFrame([], columns=['ShipName', 'VoyageFrom', 'VoyageTo', 'Nationality', 'Year',])
    for i in range(0, len(csvlogs['ShipName'])):
        if uniques[uIndex] == csvlogs['ShipName'][i]:
            newframe.loc[uIndex] = csvlogs.ix[i]

            uIndex += 1
            id +=1
            if uIndex == len(uniques):
                return newframe
        else:
            continue





#Old Wooden Ships
#Select data

import pandas as pd

def select_data(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear):
    countrieslist = list()
    
    #Determine countries we're interested in
    if isAmerican == 1:
        countrieslist.append("American")
    if isBritish == 1:
        countrieslist.append("British")
    if isDanish == 1:
        countrieslist.append("Danish")
    if isDutch == 1:
        countrieslist.append("Dutch")
    if isHamburg == 1:
        countrieslist.append("Hamburg")
    if isSpanish == 1:
        countrieslist.append("Spanish")
    if isSwedish == 1:
        countrieslist.append("Swedish")
    
   
    #Select only those countries
    data = extract_records()
    newframe = pd.DataFrame([], columns=['ShipName', 'VoyageFrom', 'VoyageTo', 'Nationality', 'Year'])
    newframe2 = pd.DataFrame([], columns=['Id', 'ShipName', 'VoyageFrom', 'VoyageTo', 'Nationality', 'Year'])
    newIndex1 = 0
    newIndex2 = 0
    for i in range(0, len(data)):
        if data['Nationality'][i] in countrieslist:
            
            newframe.loc[newIndex1] = data.ix[i]
            newIndex1 += 1
        
    #Select rows within year range
    for i in range(0, len(newframe)):
        if fromYear <= newframe['Year'][i] <= toYear:
            newframe2.loc[newIndex2] = newframe.ix[i]
            
            newframe2['Id'][newIndex2]=newIndex2
            newIndex2 += 1
    
    return newframe2



In [101]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  2 19:47:31 2015

@author: mbatra
"""

import pandas as pd
import distance
from textblob import TextBlob


#Searches the word the the list given
def searchName(word1,arr):
    lenArr = len(arr)
    index = None
    for i in range(0,lenArr ):
        strW = str(arr[i])
        strW = strW.lower()
        word1 = str(word1).lower()
        if( word1 in strW.lower()):
            global index
            index=i          
    if index == None:
        return None
    else:
        return index
    
    
#Gives the nearest matching word using Jaccard distance      
def closestWord(word, arr):
    lenArr = len(arr)
    dist = 0.0
    global finalId 
    finalId = 0
    for i in range(1,lenArr):
        temp = distance.jaccard(str(word),str(arr[i]))
        if(dist > temp):
            dist = temp
            finalId = i          
    if dist == 0:
        return None;
    else:            
        return finalId
     
     
     
#Sets the row number
def setrowNum(num):
    global rownum
    rownum = num
    return 0

    
#To the current row number        
def getrowNum():
    return rownum
    

def getLocationDFrame(vfOrvt, shipData):

    colNum = None
    if vfOrvt =='vf':
        global colNum
        colNum = 'VoyageFrom'
    else:
        global colNum
        colNum = 'VoyageTo'
        
    geoData = pd.io.parsers.read_csv("Geodata.csv",low_memory=False)
    LocationVF = pd.DataFrame([], columns=['Id','Latitude','Longitude'])
    n = len(shipData[colNum])
    rownum = 0
    
    for i in range(0,n):
        shipFrom = shipData[colNum][i]
        reslt= searchName(shipFrom,geoData['Place'])
    
        if reslt != None:
            reslt = setrowNum(reslt)
        
        elif reslt == None:
            checkModName = searchName(shipFrom,geoData['ModernName'])
            if checkModName != None:
                checkModName = setrowNum(checkModName)
           
            elif checkModName == None:
                checkAltName = searchName(shipFrom,geoData['Alternative.Name'])
                if checkAltName != None:
                    checkAltName = setrowNum(checkAltName)
               
                elif (checkAltName == None):
                    text = str(shipFrom).decode('ascii', "ignore")
                    b = TextBlob(text)
                    lang = str(b.detect_language())
                    #print lang
                    if lang !='en':
                        try:
                            #print "Entered try catch"
                            transWord=b.translate(from_lang=lang, to='en')
                            #print b.translate(from_lang=lang, to='en')
                        except:
                            transWord = text
                            #print "Cotinues"
                        checktransWord = searchName(transWord,geoData['Place'])
                        #print "checktransWord" , checktransWord
                        if checktransWord != None:
                            checktransWord = setrowNum(checktransWord)
                       
                        elif checktransWord == None:
                            lastResort= closestWord(transWord,geoData['Place']) 
                            if lastResort != None:
                                lastResort = setrowNum(lastResort)
                        
        row = getrowNum()
        print "final value of row",row ,rownum
        LocationVF = LocationVF.append({'Id':i,'Latitude':geoData['DecLatitude'][row],'Longitude':geoData['DecLongitude'][row]}, ignore_index=True)

    return LocationVF


<ipython-input-101-4a0003a0f4ad>:22: SyntaxWarning: name 'index' is assigned to before global declaration
  global index
<ipython-input-101-4a0003a0f4ad>:64: SyntaxWarning: name 'colNum' is assigned to before global declaration
  global colNum
<ipython-input-101-4a0003a0f4ad>:67: SyntaxWarning: name 'colNum' is assigned to before global declaration
  global colNum


In [ ]:
def generate_vectors(shipData):

    #retrieve to and from Lat/Lon coordinates and rename columns
    fromData = getLocationDFrame('vf', shipData)
    toData = getLocationDFrame('vt', shipData)

    fromData.columns = ['Id', 'FromLat', 'FromLon']
    toData.columns = ['Id', 'ToLat', 'ToLon']

    #join dataframes
    merged = pd.concat([shipData, fromData], axis=1, join='inner')
    merged = pd.concat([merged, toData], axis=1, join='inner')
    
    #print merged 
    
    merged = merged.replace(['American', 'British', 'Danish', 'Dutch', 'Hamburg', 'Spanish', 'Swedish'], ['blue', 'red', 'black', 'white', 'purple', 'yellow', 'green'])

    #drop unnecessary variables
    merged = merged.drop(['ID', 'ShipName', 'VoyageFrom', 'VoyageTo', 'Year', 'Id'], axis=1)
    return merged



In [140]:
#Old Wooden Ships
#Old Wooden Map

from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt


def draw_voyage_map(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear):
    print"Reached draw_voyage_map"
    print isAmerican
    data = select_data(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear)
    vectors = generate_vectors(data)
    print vectors
    
    map = Basemap(projection='robin',lon_0=0)
    map.drawcoastlines()
    map.fillcontinents(color='brown', lake_color='tan')
    map.drawmapboundary(fill_color='tan')
    map.drawparallels(np.arange(-.5,.5, 0.5))
    map.drawcountries()
    
   
    
    for i in range(0, len(vectors)):
        try:
            map.drawgreatcircle(vectors['FromLon'][i], vectors['FromLat'][i], vectors['ToLon'][i], vectors['ToLat'][i], color=vectors['Nationality'][i])
        except:
            continue
    
    plt.title("Old Wooden Ships: Voyages")
    plt.show()


In [134]:
'''
def go_function(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear):
    print fromYear
    fromYear=int(eFrom.get())
    if fromYear != 0:
        print fromYear
        print "Entered basemap call"
        draw_voyage_map(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear)
    else:
        return None
    '''

'\ndef go_function(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear):\n    print fromYear\n    fromYear=int(eFrom.get())\n    if fromYear != 0:\n        print fromYear\n        print "Entered basemap call"\n        draw_voyage_map(isAmerican, isBritish, isDanish, isDutch, isHamburg, isSpanish, isSwedish, fromYear, toYear)\n    else:\n        return None\n    '

In [151]:
#UI
from Tkinter import *


    
def main():
    window = Tk()
    window.title("Old Wooden Ships")
    
    

    
    #Frame 1: Nationality menu
    isAmerican = IntVar()
    isBritish = IntVar()
    isDanish = IntVar()
    isDutch = IntVar()
    isHamburg = IntVar()
    isSpanish = IntVar()
    isSwedish = IntVar()
    
    frame1 = Frame(window)
    frame1.pack(side=LEFT)
    label1 = Label(frame1, text="Select Nationalities: ")
    label1.pack()
    chAmerican = Checkbutton(frame1, text="American (Blue)", variable=isAmerican, onvalue=1, offvalue=0, anchor='w')
    chAmerican.pack()
    chBritish = Checkbutton(frame1, text="British (Red)", variable=isBritish, onvalue=1, offvalue=0, anchor='w')
    chBritish.pack()
    chDanish = Checkbutton(frame1, text="Danish (Black)", variable=isDanish, onvalue=1, offvalue=0, anchor='w')
    chDanish.pack()
    chDutch = Checkbutton(frame1, text="Dutch (White)",variable=isDutch , onvalue=1, offvalue=0, anchor='w')
    chDutch.pack()
    chHamburg = Checkbutton(frame1, text="Hamburg (Purple)", variable=isHamburg , onvalue=1, offvalue=0, anchor='w')
    chHamburg.pack()
    chSpanish = Checkbutton(frame1, text="Spanish (Yellow)", variable=isSpanish, onvalue=1, offvalue=0, anchor='w')
    chSpanish.pack()
    chSwedish = Checkbutton(frame1, text="Swedish (Green)", variable=isSwedish, onvalue=1, offvalue=0, anchor='w')
    chSwedish.pack()
    
    
    def go_function():
        
        fromYear=int(eFrom.get())
        print fromYear
        toYear = int(eTo.get())
        isAmerican1=isAmerican.get()
        print isAmerican
        isBritish1= isBritish.get()
        isDanish1=isDanish.get()
        isDutch1=isDutch.get()
        isSpanish1=isSpanish.get()
        isSwedish1=isSwedish.get()
        p
        if fromYear != 0:
            print fromYear
            print "Entered basemap call"
            draw_voyage_map(isAmerican1, isBritish1, isDanish1, isDutch1, isHamburg1, isSpanish1, isSwedish1, fromYear, toYear)
        else:
            return None
    
    
    #Frame 2: Year Selection
    frame2 = Frame(window)
    frame2.pack(side=LEFT)
    label2 = Label(frame2, text="Enter Years:")
    label2.grid(row=1, column=1, columnspan=2)
    lFrom = Label(frame2, text="From: ", justify=LEFT)
    lFrom.grid(row=2, column=1)
    eFrom = Entry(frame2)
    eFrom.insert(0, 0)
    eFrom.grid(row=2, column=2)
    lTo = Label(frame2, text="To: ", justify=LEFT)
    
    lTo.grid(row=3, column=1)
    eTo = Entry(frame2)
    eTo.insert(0, 0)
    eTo.grid(row=3, column=2)
    

    def go_function1():
        
        return go_function()
    #window.update()
    goButton = Button(frame2, text="GO", command=go_function1)
    
    

    goButton.grid(row=4, column=2)
    
    

    
    window.mainloop()
    
    
main()

1750
PY_VAR350
1750
Entered basemap call


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Benjamin\Anaconda\lib\lib-tk\Tkinter.py", line 1536, in __call__
    return self.func(*args)
  File "<ipython-input-151-19e8fde94763>", line 82, in go_function1
    return go_function()
  File "<ipython-input-151-19e8fde94763>", line 57, in go_function
    draw_voyage_map(isAmerican1, isBritish1, isDanish1, isDutch1, isHamburg1, isSpanish1, isSwedish1, fromYear, toYear)
NameError: global name 'isHamburg1' is not defined
